# Background and Motivation

If you're like me, you've been learning Python for a little while now in the hopes of landing a data analyst/data scientist job. You've been following online tutorials, reading your data from downloaded csv's, data-munging, modeling and plotting entirely within a jupyter notebook. 

The problem is this is not a realistic data analysis workflow. Real data doesn't live in flat files that are easily read into memory. So I decided to start a project that would be a closer approximation to what a data anlayst actually does. This project will require me to create and manage a database, build a data pipeline to populate that database from an api, and finally, build a small web app that hosts an analytical dashboard for visualization.

Finally, it's always more fun to work on a problem that is more interesting to you. I really enjoy running, so the idea of working with Strava data was pretty exciting for me.

I will blog my way through this project. As I learn and code up projects, I generally will depart from the main project and explore a concept in isolation (how does list comprehension work?) and then return to the main problem. I imagine that my posts will follow this format; posts which walks through how the final result was built, and spurs along the way that help build out a deeper understanding of how pieces work.

# What this post covers

This is a short post that explores the following points regarding interacting and managing PostgreSQL from Python. These are the points, thus far, that I think are necessary to expore further to build understanding. You can easily skip the first part on background definitions and go straight to "2. Driving PostgreSQL from Python"

1. Some background definitions
    1. What is a database?
    2. What is a relational Databases
    3. What is SQL?
    4. What is PostgreSQL?
    5. What is SQLAlchemy?
2. Driving PostgreSQL from Python
    1. Create a table
    2. Retrieve information about the table
    3. List what tables are in the database
    4. Delete a table
    5. Insert data into table
    6. Delete data from a table
    7. Update data
3. How do we avoid inserting duplicates?
    1.

# Some background definitions

## What is a Database?

A **database,** or **database system** generally consists of 3 things; a **database**, a **database management system** and a **database model**. I think the analogy of a library helps put this into real-world context. 

The library is composed of all the books, newspapers, magazines etc stored on shelves organized systematically. This is the **database**. However, customers can't access the library without the library staff and the procedures they bring to the physical library. Customers use the library staff and their procedures for finding, checking out, returning books. This is the role of the **database management system**. Finally, the **database model** defines what type of data is stored and how that data is organized. Most libraries use the dewey decimal system as a model for storing and organizing books/newspapers etc.


## Relational Databases
Relational Databases make up the majority of the database systems that are used today. An extremely simplified explanation of a relational database is that it is a collection of named tables, or flat csv files, holding information. We can define how many columns each table has and what type of data each column can hold. Even further, we can create dependencies between the tables, such that the value of column in one table must be found in the column of another table.

There are some alternatives to the relational model, however for our purposes, we will stick with the concept of a collection of tables.

## What is SQL?
At it's essence, SQL is a language specific to querying data from a relational database system. It is a **declarative language** which means that it describes what to do, not how it needs to be done. Using the library example, you would tell the librarian "Can you bring me 'SQL for Dummies'?" - you would not continue give them additional instructions on exactly how to search for and retrieve the book.


#### Structure of SQL
There are four main operations that SQL performs - query the data, manipulate the data, define the data/schema (such as tables, data types, constraints), and define access/control of data (who can perform the other three). I will refrain from attempting to break down the language elements any further at this point.

#### Quick aside on syntax
Syntax means the arrangment of words to create well formed sentences (to hopefully express meaning). In SQL we can think of syntax as the how we write and what we write to achieve a desired interaction with the database. Unfortunately the syntax is not consistent across database systems, however the syntax (I think) is similar enough that one should be able to pick up new flavors of SQL relatively quickly.

## What is PostgreSQL
PostgreSQL is an open-source database system. It's great because it's free to download and easy to work with (so far). With it, we can create a database locally on our computers and fairly quickly interact with it in our python script.

## What is SQLAlchemy
SQLAlchemy is a toolkit that allows us to interact with our database using python. With SQLAlchemy, we can connect to our PostgreSQL database and start experimenting with the four main operations (query, manipulate, define data/schema, define access and control). Because we want to build experience and practice writing queries, SQLAlchemy will let us pass queries written in SQL to the Postgres database.

There is a lot more that SQLAlchemy can do, check out this article for more detail https://www.compose.com/articles/using-postgresql-through-sqlalchemy/


# Driving PostgreSQL with SQLAlchemy

Note that I previously installed PostgreSQL. Download instructions can be found from their website.

In [1]:
import pandas as pd
import numpy as np
import psycopg2 # required to use sqlalchemy with PostgreSQL
from sqlalchemy import create_engine # also used to communicate with postgres

/Users/jakekirsch/.pyenv/versions/jupyter3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


`create_engine` creates an `engine` object, which abstracts away all the complexity of managing a connection to a database. We can connect to many types of databases using SQLAlchemy and the engine construct, in our example we connect to PosgreSQL.

The first part of the string `'postgresql:'` defines the `Dialect` for the engine, or the type of database we're connecting to. The remainder of the string defines the location of that server.

In [2]:
# we can use SQLAlchemy to create an engine that allows us to interact with our Postgres DB with SQL code
engine = create_engine('postgresql://jakekirsch:@localhost/jakekirsch')

## Create a new table

Now, we need to build a simple table to get started. I'm going to hand craft all of the db interactions because I would like to practice the SQL syntax. To do so, we use the `.execute()` function of our `engine`, which accepts strings that are syntactically correct SQL statements. 

The great thing about SQL is that it's pretty easy to read. As you might guess, the first statement tells my db to drop a table named `test_table` if it exists.

The second statement is everything that comes after `CREATE TABLE`. Here we are defining a table with the name `test_table` that has the column names `alpha`, `beta` and `gamma`. After each column name, we define the data type; `bigint`, `int`, and `text` respectively. 

In [3]:
# engine was created before 
# create a simple table
engine.execute("""DROP TABLE IF EXISTS test_table;

CREATE TABLE test_table (
alpha bigint,
beta int,
gamma text);""")

## Retrieve information about newly created table
Now we might want to confirm that it was successfully created. To check from within this script, we can use the following statement.

#### Aside on SCHEMA
A schema is a level of organization within a database, at this point we can think of them like folders on your computer. Schema's contain tables, as well as other database objects that we may have created.

The `INFORMATION_SCHEMA` is a schema that is in evey database by default. The tables and objects in the `INFORMATION_SCHEMA` give us access to information about the structure of our database. We can use the table `COLUMNS` in the `INFORMATION_SCHEMA` to obtain information about our newly created table. In our case, we want to see the column names and data types of `test_table`

#### Access the information returned by a query
We have to assign the `engine.execute()` call to a variable in order to access the information that is returned by our query. A `ResultProxy` abstracts away the management of lower level objects that interact with our database. At this point, if I'm executing a query that should return information (table), as opposed to manipulate or update information, I assign to a variable.

We then use `.fetchone()` or `.fetchall()` to access our information

In [4]:
# confirm it was created
col_info = """SELECT column_name, data_type, character_maximum_length
    FROM INFORMATION_SCHEMA.COLUMNS where table_name = 'test_table';"""
result = engine.execute(col_info)
result.fetchall()

[('alpha', 'bigint', None), ('beta', 'integer', None), ('gamma', 'text', None)]

Great, the results are as expected. 

What is interesting, is that once you exhaust the information that the `result` variable contains (i.e. the rows that are in the returned table) `result` is empty, in the sense you cannot access the rows again unless you execute the query once more.

In [5]:
result.fetchall()

[]

## List what tables are in the database

Perhaps your new to working on this database, or you forget what you named certain tables, we can use the `INFORMATION_SCHEMA` to obtain all of the tables in the database. We access this information in the `INFORMATION_SCHEMA.tables` relation. First let's see what columns are in this `INFORMATION_SCHEMA.tables` relation

In [6]:
col_info_tables = '''
SELECT column_name, data_type, character_maximum_length FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'tables';
'''
engine.execute(col_info_tables).fetchall()

[('table_catalog', 'character varying', None),
 ('table_schema', 'character varying', None),
 ('table_name', 'character varying', None),
 ('table_type', 'character varying', None),
 ('self_referencing_column_name', 'character varying', None),
 ('reference_generation', 'character varying', None),
 ('user_defined_type_catalog', 'character varying', None),
 ('user_defined_type_schema', 'character varying', None),
 ('user_defined_type_name', 'character varying', None),
 ('is_insertable_into', 'character varying', 3),
 ('is_typed', 'character varying', 3),
 ('commit_action', 'character varying', None)]

So we'll retrieve the first 4 to make it easier to read

In [7]:
list_tables = """
SELECT table_catalog, table_schema, table_name, table_type FROM INFORMATION_SCHEMA.tables
"""
engine.execute(list_tables).fetchall()

[('jakekirsch', 'public', 'test_table', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_statistic', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_foreign_table', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_authid', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_shadow', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_roles', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_settings', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_file_settings', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_hba_file_rules', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_config', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_user_mapping', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_replication_origin_status', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_subscription', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_statio_all_indexes', 'VIEW'),
 ('jakekirsch', 'pg_catalog', 'pg_largeobject', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_type', 'BASE TABLE'),
 ('jakekirsch', 'pg_catalog', 'pg_attribute', 'BASE

Wow, lot of tables here. To see just the tables that we have just created, we need to restrict the to the database tables that are in the `'public'` schema. The `public` schema is created by default, and tables are added to this schema unless we specify otherwise.

In [8]:
public_tables = '''
SELECT * FROM INFORMATION_SCHEMA.tables WHERE table_schema = 'public'
'''
engine.execute(public_tables).fetchall()

[('jakekirsch', 'public', 'test_table', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)]

Great, we see our `test_table` right there

## Delete (drop) a table 

Suppose we want to completely delete our `test_table`. If we are the owner of said table, we can do the following

In [9]:
drop_table = """DROP TABLE test_table;"""
engine.execute(drop_table)

And then check the public schema for a list of tables

In [10]:
res = engine.execute('''SELECT * from INFORMATION_SCHEMA.tables WHERE table_schema = 'public'; ''')
res.fetchall()

[]

Excellent, table removed. 

## Insert data into table

Now we want to practice inserting some data into our newly created table. We'll experiment with a few methods on this, we'll re-create the sample `test_table` first and a dataframe with 3 columns.

In [11]:
# create a simple table
engine.execute("""DROP TABLE IF EXISTS test_table;

CREATE TABLE test_table (
alpha bigint,
beta int,
gamma text);""")

#### Using SQL statements

This is the basic structure of `INSERT` - we define the table `test_table` that we want to insert data into, the columns `(alpha, beta, gamma)` of that table we want to insert into, and then define the data using `VALUES (1, 2, 'this')`.


In [12]:
insert_some_data = """
INSERT INTO test_table (alpha, beta, gamma)
VALUES (1, 2, 'this');
"""
engine.execute(insert_some_data)

In [13]:
engine.execute("SELECT * FROM test_table;").fetchall()

[(1, 2, 'this')]

If we want to enter more than a single row, we simply add another tuple after `VALUES`

In [14]:
insert_more_data = """
INSERT INTO test_table (alpha, beta, gamma)
VALUES 
(1, 2, 'row1'),
(3, 4, 'row2'),
(6, 5, 'row3');
"""
engine.execute(insert_more_data)
engine.execute("SELECT * FROM test_table;").fetchall()

[(1, 2, 'this'), (1, 2, 'row1'), (3, 4, 'row2'), (6, 5, 'row3')]

Let's experiment with missing values. The missing value in Postgres is `NULL` however it's `None` in python.

So if we are passing a null value to Postgres from Python, we need to keep in mind that `None` needs to be `NULL` in the SQL statement. When we retrieve the values, it is `None` in python 

In [15]:
insert_test_one = """
INSERT INTO test_table (alpha, beta, gamma)
VALUES
(10, 10, None);"""

try:
    engine.execute(insert_test_one)
    print("Successfully inserted None")
except:
    print("Can't insert None")
    
insert_test_two = """
INSERT INTO test_table (alpha, beta, gamma)
VALUES
(10, 10, NULL);"""

try:
    engine.execute(insert_test_two)
    print("Successfully inserted NULL")    
except:
    print("Can't insert NULL")
    


Can't insert None
Successfully inserted NULL


In [16]:
engine.execute("SELECT * FROM test_table;").fetchall()

[(1, 2, 'this'),
 (1, 2, 'row1'),
 (3, 4, 'row2'),
 (6, 5, 'row3'),
 (10, 10, None)]

Let's also experiment with defining different column names. If we omit the column names, what happens? Well according to the documentation, the implied columns are the column names in their declared order, so we would expect our value tuple to pass the first value into the `alpha` column, second value into `beta` etc.

In [17]:
engine.execute("""
INSERT INTO test_table
VALUES (12,12,'no_col');
""")

engine.execute("SELECT * FROM test_table;").fetchall()

[(1, 2, 'this'),
 (1, 2, 'row1'),
 (3, 4, 'row2'),
 (6, 5, 'row3'),
 (10, 10, None),
 (12, 12, 'no_col')]

Great, works as expected. Now what happens if we define a value tuple that is smaller than the number of columns? The documentation says that columns (explicit or implied - written after `test_table` or not) not supplied (by a value in the tuple) will be filled by the default value specified for that column. If no default value is specified then it will fill with Null which what we see here. Note that the default value for a column is specified during the `CREATE TABLE` statement

In [18]:
engine.execute("""
INSERT INTO test_table
VALUES (12, 12);
""")

engine.execute("SELECT * FROM test_table;").fetchall()

[(1, 2, 'this'),
 (1, 2, 'row1'),
 (3, 4, 'row2'),
 (6, 5, 'row3'),
 (10, 10, None),
 (12, 12, 'no_col'),
 (12, 12, None)]

Finally, le'ts see what happens when we pass in different values. The below example fails, as one would expect

In [19]:
try:
    engine.execute("""
    INSERT INTO test_table
    VALUES ('text','12', 'this value');
    """)
    engine.execute("SELECT * FROM test_table;").fetchall()
except:
    print("Can't insert this value")

Can't insert this value


However this example doesn't, it seems like Postgres attempts to coerce the tuple to the correct data type.

In [20]:
engine.execute("""
INSERT INTO test_table
VALUES (1, '12', 'this second value');""")

engine.execute("SELECT * FROM test_table;").fetchall()

[(1, 2, 'this'),
 (1, 2, 'row1'),
 (3, 4, 'row2'),
 (6, 5, 'row3'),
 (10, 10, None),
 (12, 12, 'no_col'),
 (12, 12, None),
 (1, 12, 'this second value')]

#### SQL statements from dataframe

How would we use the SQL statement to insert data that is in a dataframe? A rudimentary approach is to loop over the dataframe, formatting an SQL statement that will insert the data as we progress through the loop

In [75]:
# create a sample df
alpha = np.random.normal(loc=50, scale=20, size=100)
beta = np.random.uniform(low=0, high=1, size=100)
gamma = np.random.wald(mean=25, scale=10, size=100)

data = {'alpha':alpha,
       'beta':beta,
       'gamma':gamma}

df = pd.DataFrame(data)

In [76]:
# insert the values
for index, row in df.loc[0:2,].iterrows():
    engine.execute("""INSERT INTO test_table
    VALUES ({}, {}, {});""".format(row['alpha'], row['beta'], row['gamma']))

In [77]:
len(engine.execute("SELECT * FROM test_table;").fetchall()) # data was successfully added

9

In [78]:
# we'll address this later on, but cleaning up the table to make it easier to work with
engine.execute("DELETE FROM test_table;") 

Another way is to use a parameterized query, where we use `%s` operators as placeholders. Then we pass a tuple of values as an argument to the execute() function. The result is that the values in the tuple are inserted into the query statement

In [79]:
value_tuple = (1, 2, "test")

engine.execute("""
INSERT INTO test_table
VALUES (%s, %s, %s);""", value_tuple)

engine.execute("""
SELECT * FROM test_table;""").fetchall()

[(1, 2, 'test')]

Let's try this with a list of tuples

In [80]:
values_tuple_list = [(1,2,'second'),
                    (3, 4, 'third')]

engine.execute("""INSERT INTO test_table VALUES (%s, %s, %s);""", values_tuple_list)

engine.execute("""SELECT * FROM test_table;""").fetchall()

[(1, 2, 'test'), (1, 2, 'second'), (3, 4, 'third')]

Great, that works beautifully. If we convert our dataframe to a list of tuples, we should be able to use this approach

In [83]:
value_tuple_list = list(df.itertuples(index=False))

In [84]:
engine.execute("""INSERT INTO test_table VALUES (%s, %s, %s);""", value_tuple_list)
engine.execute("""SELECT * FROM test_table LIMIT 10;""").fetchall()

[(1, 2, 'test'),
 (1, 2, 'second'),
 (3, 4, 'third'),
 (66, 0, '4.936065031893158'),
 (54, 0, '87.82269591642915'),
 (26, 1, '43.89690269150002'),
 (24, 0, '8.479785634961758'),
 (71, 1, '24.70559751374415'),
 (37, 1, '54.41480727753379'),
 (36, 1, '7.7457648294183805')]

#### Insert using Pandas

Pandas also offers the ability to interact with a database using `to_sql` and `read_sql` which can make moving data between flat files --> pandas --> db and db --> pandas --> plots very easy

In [93]:
pd.read_sql('SELECT * FROM test_table LIMIT 10;', con = engine)

,alpha,beta,gamma
0,1,2,test
1,1,2,second
2,3,4,third
3,66,0,4.936065031893158
4,54,0,87.82269591642915
5,26,1,43.89690269150002
6,24,0,8.479785634961758
7,71,1,24.70559751374415
8,37,1,54.41480727753379
9,36,1,7.7457648294183805


In [85]:
with engine.connect() as conn:
    df.to_sql('test_table', con=conn, if_exists='append', index=False)

In [86]:
col_info = """SELECT column_name, data_type, character_maximum_length
    FROM INFORMATION_SCHEMA.COLUMNS where table_name = 'test_table';"""

with engine.connect() as conn:
    res = pd.read_sql(col_info, con = conn)
res

,column_name,data_type,character_maximum_length
0,alpha,bigint,None
1,beta,integer,None
2,gamma,text,None


In [24]:
with engine.connect() as conn:
    res = pd.read_sql("select * from test_table limit 5", con = conn)
res.dtypes

alpha     int64
beta      int64
gamma    object
dtype: object

In [25]:
# now add new text values
alpha = np.random.normal(loc=50, scale=20, size=100)
beta_text = np.random.uniform(low=0, high=1, size=100).astype(str)
gamma = np.random.wald(mean=25, scale=10, size=100)

data = {'alpha':alpha,
       'beta':beta_text,
       'gamma':gamma}

df = pd.DataFrame(data)

In [26]:
# append again, note that the column names must line up
with engine.connect() as conn:
    df.to_sql('test_table', con=conn, if_exists='append', index=False)

DataError: (psycopg2.DataError) invalid input syntax for integer: "0.8458343519939661"
LINE 1: ...e (alpha, beta, gamma) VALUES (43.21666123723385, '0.8458343...
                                                             ^
 [SQL: 'INSERT INTO test_table (alpha, beta, gamma) VALUES (%(alpha)s, %(beta)s, %(gamma)s)'] [parameters: ({'alpha': 43.21666123723385, 'beta': '0.8458343519939661', 'gamma': 4.919045837851243}, {'alpha': 42.0928934685868, 'beta': '0.09803323934637231', 'gamma': 4.535407175754884}, {'alpha': 18.034655821333224, 'beta': '0.25012919305514003', 'gamma': 18.361464172281284}, {'alpha': 50.93536942470595, 'beta': '0.618586703466302', 'gamma': 10.526746363084358}, {'alpha': 72.33369603789404, 'beta': '0.0738346496519855', 'gamma': 21.166475412254098}, {'alpha': 46.41056670930328, 'beta': '0.6741765914178779', 'gamma': 17.59893937696004}, {'alpha': 35.466527245435174, 'beta': '0.3233499618286837', 'gamma': 22.68236250285856}, {'alpha': 60.40460759497108, 'beta': '0.620109

as expected, we couldn't pass in values of text into the database